Program, ki iz datoteke neprevedeniSloStavki.npy prebere stavke, potem pa naključno izibra določeno število stavkov in jih prevaja.

In [2]:
import datetime
import numpy as np
import random
import requests
from requests.structures import CaseInsensitiveDict
import deepl
import datetime
import numpy as np

import googletrans
from googletrans import Translator
from json import JSONDecodeError

# Clarin
url = "http://localhost:4000/api/translate"
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"

# Google
translatorGoogle = Translator()

# Deepl
auth_key = "0237a978-df9a-da8b-275a-5b664b40feb0:fx"  # Replace with your key
translatorDeepl = deepl.Translator(auth_key)

steviloStavkovZaPrevod = 10

In [3]:
# Slovar oznak za shranjevanje v datoteko prevedeniStavki.npy
# Oznaka je sestavljena iz postopka prevajanje in prevajalca, ki je prevajal
# Primer SLO_AN_SLO_GOOGLE_DEEPL pomeni, da je najprej prevedel Google iz slovenščine v angleščino
#    potem pa je Deepl prevedel iz angleščine v slovenščino 

slovar = dict({"original":0, 
               "SLO_AN_CLARIN":1, "SLO_AN_GOOGLE":2, "SLO_AN_DEEPL":3, 
               "SLO_AN_SLO_CLARIN_CLARIN":4, "SLO_AN_SLO_CLARIN_GOOGLE":5, "SLO_AN_SLO_CLARIN_DEEPL":6,
               "SLO_AN_SLO_GOOGLE_CLARIN":7, "SLO_AN_SLO_GOOGLE_GOOGLE":8, "SLO_AN_SLO_GOOGLE_DEEPL":9,
               "SLO_AN_SLO_DEEPL_CLARIN":10, "SLO_AN_SLO_DEEPL_GOOGLE":11, "SLO_AN_SLO_DEEPL_DEEPL":12
              })

In [4]:
# prebere še neprevedene stavke
neprevedeniStavki = np.load('neprevedeniSloStavki.npy',allow_pickle=True).tolist()

In [5]:
# preberem že prevedene stavke, če obstajajo
try:
    tabelaPrevedenihStavkov = np.load('prevedeniStavki.npy',allow_pickle=True).tolist()
except:
    print("Ni še datoteke s prevedenimi stavki")
    tabelaPrevedenihStavkov = []

In [6]:
# Število neprevedenih in prevedenih stavkov
print(len(neprevedeniStavki))
print(len(tabelaPrevedenihStavkov))

6714971
15067


In [9]:
zacetek = datetime.datetime.now()
for i in range(steviloStavkovZaPrevod):
    
    # Izbira indeksa neprevedenega stavka
    indeks = random.randrange(len(neprevedeniStavki))
    stavekZaPrevod = neprevedeniStavki[indeks]
    
    print(i)
    print(stavekZaPrevod)
    
    # preveri, da ima stavek med 5 in 20 presledkov, kar pomeni, da naj bi ieml vsaj 5 in ne več kot 20 besed
    if stavekZaPrevod.count(' ') > 20 or stavekZaPrevod.count(' ') < 5:
        print("Stavek je prevelik ali premajhen")
        print()
        continue
    
    # spremenljivke za prevedene stavke
    stavekSloAnClarin = ""
    stavekSloAnSloClarinClarin = ""
    stavekSloAnGoole = ""
    stavekSloAnSloGoogleGoogle = ""
    stavekSloAnDeepl = ""
    stavekSloAnSloDeeplDeepl = ""
    stavekSloAnSloGoogleClarin = ""
    stavekSloAnSloClarinGoogle = ""
    stavekSloAnSloClarinDeepl = ""
    stavekSloAnSloGoogleDeepl = ""
    stavekSloAnSloDeeplGoogle = ""
    stavekSloAnSloDeeplClarin = ""
    
    # vrstica s stavki, ki se doda v tabelo prevedenih stavkov
    vrstica = [""]*13
    vrstica[0] = stavekZaPrevod
    
    # prevod iz slovenščine v angleščino preko clarina v angleščino in potem iz angleščine nazaj v slovenščino preko Clarin
    try:
        data = '{ "src_language":"sl","tgt_language":"en","text":"'+stavekZaPrevod+'"}'
        data = data.encode("utf-8")
        resp = requests.post(url, headers=headers, data=data)
        stavekSloAnClarin = eval(resp.text)['result']
        vrstica[slovar["SLO_AN_CLARIN"]] = stavekSloAnClarin
        data = '{ "src_language":"en","tgt_language":"sl","text":"'+stavekSloAnClarin+'"}'
        data = data.encode("utf-8")
        resp = requests.post(url, headers=headers, data=data)
        stavekSloAnSloClarinClarin = eval(resp.text)['result']
        vrstica[slovar["SLO_AN_SLO_CLARIN_CLARIN"]] = stavekSloAnSloClarinClarin
    except:
        print("Nekaj šlo narobe prevajanje CLARIN")
        print(resp)
        
    #print("Prevod CLARIN SLO AN",stavekSloAnClarin)
    #print("Prevod CLARIN CLARIN SLO AN SLO",stavekSloAnSloClarinClarin)
    #print()
        
    
        
    # prevod iz slovenščine v angleščino preko Google v angleščino in potem iz angleščine nazaj v slovenščino preko google
    try:
        result = translatorGoogle.translate(stavekZaPrevod, src='sl', dest='en')
        stavekSloAnGoole = result.text
        vrstica[slovar["SLO_AN_GOOGLE"]] = stavekSloAnGoole
        result = translatorGoogle.translate(stavekSloAnGoole, src='en', dest='sl')
        stavekSloAnSloGoogleGoogle = result.text
        vrstica[slovar["SLO_AN_SLO_GOOGLE_GOOGLE"]] = stavekSloAnSloGoogleGoogle
    except:
        print("Nekaj šlo narobe prevajanje GOOGLE")
        
    #print("Prevod GOOGLE SLO AN",stavekSloAnGoole)
    #print("Prevod GOOGLE GOOGLE SLO AN SLO",stavekSloAnSloGoogleGoogle)
    #print()
        
        
    """    
    # prevod iz slovenščine v angleščino preko Deepl v angleščino in potem iz angleščine nazaj v slovenščino preko Deepl
    try:
        result = translatorDeepl.translate_text(stavekZaPrevod, source_lang="SL", target_lang="EN-GB")
        stavekSloAnDeepl = result.text
        vrstica[slovar["SLO_AN_DEEPL"]] = stavekSloAnDeepl
        result = translatorDeepl.translate_text(stavekSloAnDeepl, source_lang="EN", target_lang="SL")
        stavekSloAnSloDeeplDeepl = result.text
        vrstica[slovar["SLO_AN_SLO_DEEPL_DEEPL"]] = stavekSloAnSloDeeplDeepl
    except:
        print("Nekaj šlo narobe prevajanje DEEPL")
        
    #print("Prevod DEEPL SLO AN",stavekSloAnDeepl)    
    #print("Prevod DEEPL DEEPL SLO AN SLO",stavekSloAnSloDeeplDeepl)
    #print()
    """
    
        
    # prevod angleškega stavka prevedenega preko Google v slovenščino preko Clarin
    try:
        data = '{ "src_language":"en","tgt_language":"sl","text":"'+stavekSloAnGoole+'"}'
        data = data.encode("utf-8")
        resp = requests.post(url, headers=headers, data=data)
        stavekSloAnSloGoogleClarin = eval(resp.text)['result']
        vrstica[slovar["SLO_AN_SLO_GOOGLE_CLARIN"]] = stavekSloAnSloGoogleClarin
    except:
        print("Nekaj šlo narobe prevajanje GOOGLE - CLARIN")
        
    #print("Prevod GOOGLE CLARIN SLO AN SLO",stavekSloAnSloGoogleClarin)
    #print()
        
        
    # prevod angleškega stavka prevedenega preko Clarin v slovenščino preko Google
    try:
        result = translatorGoogle.translate(stavekSloAnClarin, src='en', dest='sl')
        stavekSloAnSloClarinGoogle = result.text
        vrstica[slovar["SLO_AN_SLO_CLARIN_GOOGLE"]] = stavekSloAnSloClarinGoogle
    except:
        print("Nekaj šlo narobe prevajanje CLARIN - GOOGLE")
      
    #print("Prevod CLARIN GOOGLE SLO AN SLO",stavekSloAnSloClarinGoogle)
    #print()
        
        
    """    
    # prevod angleškega stavka prevedenega preko Clarin v slovenščino preko Deepl
    try:
        result = translatorDeepl.translate_text(stavekSloAnClarin, source_lang="EN", target_lang="SL")
        stavekSloAnSloClarinDeepl = result.text
        vrstica[slovar["SLO_AN_SLO_CLARIN_DEEPL"]] = stavekSloAnSloClarinDeepl
    except:
        print("Nekaj šlo narobe prevajanje CLARIN DEEPL")
        
    #print("Prevod CLARIN DEEPL SLO AN SLO",stavekSloAnSloClarinDeepl)
    #print()
    """
    
    """
    # prevod angleškega stavka prevedenega preko Google v slovenščino preko Deepl
    try:
        result = translatorDeepl.translate_text(stavekSloAnGoole, source_lang="EN", target_lang="SL")
        stavekSloAnSloGoogleDeepl = result.text
        vrstica[slovar["SLO_AN_SLO_GOOGLE_DEEPL"]] = stavekSloAnSloGoogleDeepl
    except:
        print("Nekaj šlo narobe prevajanje GOOGLE DEEPL")
        
    #print("Prevod GOOGLE DEEPL SLO AN SLO",stavekSloAnSloGoogleDeepl)
    #print()
    """
    
    """
    # prevod angleškega stavka prevedenega preko Deepl v slovenščino preko Google
    try:
        result = translatorGoogle.translate(stavekSloAnDeepl, src='en', dest='sl')
        stavekSloAnSloDeeplGoogle = result.text
        vrstica[slovar["SLO_AN_SLO_DEEPL_GOOGLE"]] = stavekSloAnSloDeeplGoogle
    except:
        print("Nekaj šlo narobe prevajanje DEEPL - GOOGLE")
        
    #print("Prevod DEEPL GOOGLE SLO AN SLO", stavekSloAnSloDeeplGoogle)
    #print()
    """
    
    """
    # prevod angleškega stavka prevedenega preko Deepl v slovenščino preko Clarin
    try:
        data = '{ "src_language":"en","tgt_language":"sl","text":"'+stavekSloAnDeepl+'"}'
        data = data.encode("utf-8")
        resp = requests.post(url, headers=headers, data=data)
        stavekSloAnSloDeeplClarin = eval(resp.text)['result']
        vrstica[slovar["SLO_AN_SLO_DEEPL_CLARIN"]] = stavekSloAnSloDeeplClarin
    except:
        print("Nekaj šlo narobe prevajanje DEEPL - CLARIN",stavekSloAnSloDeeplClarin)
        
    #print("Prevod DEEPL CLARIN SLO AN SLO",stavekSloAnSloDeeplClarin)
    #print()
    """
    
    print()
    
    neprevedeniStavki.pop(indeks)
    tabelaPrevedenihStavkov.append(vrstica)
    
konec = datetime.datetime.now()
print(konec-zacetek)

0
5. delov sistema za uravnavanje izpustov.

1
Vir: Vode i ribe Jugoslavije, Zagreb 1932

2
Če je samo eden od staršev kadilec, obstaja možnost, da bo začel kaditi tudi otrok.

3
FDS/panorama-druzinsko ucenje
Stavek je prevelik ali premajhen

4
Priznati moram, da sem imel precej sreče s štartno številko, saj sem štartal za Belorusom Vadimom Sašurinom in pred Norvežanom Frodejem Andresenom, ki sodita v svetovni vrh biatlona.
Stavek je prevelik ali premajhen

5
Napadalci bodo bolj ustvarjalni, nevarni, manj predvidljivi.

6
To je domače vendarle prebudilo in po izkoriščeni najstrožji kazni so prek svojega najboljšega strelca Tomaža Mihelčiča znižali na 3:5.

7
1000 m z mesta: 36,3
Stavek je prevelik ali premajhen

8
Te bi bile potem vrnjene v Iran, kjer bi z njimi oskrbovali raziskovalni medicinski reaktor, s pomočjo katerega proizvajajo izotope za boj proti raku.
Stavek je prevelik ali premajhen

9
STOCKHOLM, 18. - Nogometaši Primorja so včeraj dopotovali v švedsko prestolnico in zvečer

In [10]:
# Število neprevedenih in prevedenih stavkov
print(len(neprevedeniStavki))
print(len(tabelaPrevedenihStavkov))

6714953
15085


In [11]:
# Shranjevanje skupine neprevedenih stavkov
np.save('neprevedeniSloStavki',np.asanyarray(neprevedeniStavki,dtype=object))

In [12]:
# Shranjevanje skupine prevedenih stavkov
np.save('prevedeniStavki',np.asanyarray(tabelaPrevedenihStavkov,dtype=object))